In [1]:
#https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm

In [2]:
import pandas as pd

In [3]:
#calendário de eventos corporativos
url="https://www.b3.com.br/data/files/F5/E5/E7/3A/0EB2E7108BD66BD7AC094EA8/Cronograma%20de%20Eventos%20Corporativos%202022.xlsx"

In [4]:
base=pd.read_excel(url)
atualizacao_B3=base.columns[28]
print(f"Última atualização pela B3: {atualizacao_B3}")

Última atualização pela B3: 2022-09-23 00:00:00


In [5]:
#print(base.isnull().sum())
print(f"Nessa base da B3 temos {base.shape[0]} observações e {base.shape[1]} atributos")

Nessa base da B3 temos 289 observações e 29 atributos


In [6]:
#apagando as linhas sujas
base.drop([0,1], axis=0, inplace=True)

In [7]:
#Converter as colunas de object para seus valores corretos
#base.iloc[:,i]=pd.to_datetime(base.iloc[:,i])
#base.head()

# Nosso objetivo é analisar as datas de resultados do 2° Trimestre de 2022

In [8]:
nome=base.iloc[:,0]
segmento=base.iloc[:,1]
#Resultados do 2° trimestre
previsao=base.iloc[:,14]
entrega=base.iloc[:,15]
#Nosso DataFrime
DF2tri=pd.DataFrame({
    'Nome':nome,
    'Governanca_B3':segmento,
    'Previsao':previsao,
    'Entrega':entrega})
#DF2tri.info()

In [9]:
#Infelizmente temos nas duas colunas de datas uma verdadeira bagunça!
#Alguns valores são object e outros são datetime. Alguns estão no formato do Brasil e outros não
#Vamos converter para que todos os valores fiquem no mesmo formato YYYY-MM-DD

def conversor_datas(coluna):
    dados=list(DF2tri[coluna])
    datas=[str(item) for item in dados]
    data_limpa=[]
    for i in datas:
        if len(i)==3:
               data_limpa.append(i) 
        elif i[4]=="-":
                data_limpa.append(i)
        else:
                ano=i[6:]#ano
                mes=i[3:5]#mes
                dia=i[:2]#dia
                data_limpa.append(ano+'-'+mes+'-'+dia)
    return data_limpa

#data_limpa[0:9]

In [10]:
data_prevista=conversor_datas("Previsao")
data_entrega=conversor_datas("Entrega")

DF2tri['Previsao']=data_prevista
DF2tri['Entrega']=data_entrega

#ajustando os tipos de dados em nosso DataFrame
#Youtube: https://www.youtube.com/watch?v=CHQXsSR3CkA
DF2tri["Previsao"]=pd.to_datetime(DF2tri["Previsao"])
DF2tri["Entrega"]=pd.to_datetime(DF2tri["Entrega"])

#DF2tri.head()

In [11]:
nulos=DF2tri["Entrega"].isnull().sum()
tam=DF2tri.shape[0]
print(f"Número de empresas que já divulgaram seus resultados: {tam-nulos}")

Número de empresas que já divulgaram seus resultados: 281


In [12]:
#vamos ordenar nosso DataFrame para seguir a ordem de data
#df_ordenado = df.sort_values(by='Data')
DF2tri=DF2tri.sort_values(by='Previsao')

#ordenando as empresas por ordem de resultado pelo índice
DF2tri.index=range(1,DF2tri.shape[0]+1)

#filtered_df = DF2tri[DF2tri.Previsao.eq("AERIS")]
#filtered_df
#DF2tri


# A partir daqui faremos as filtragens necessárias.

In [13]:
import datetime as dt

In [14]:
DF2tri["Ano_prev"]=DF2tri['Previsao'].dt.year
DF2tri["Mes_prev"]=DF2tri['Previsao'].dt.month
DF2tri["Dia_prev"]=DF2tri['Previsao'].dt.day
DF2tri.loc[0:5]
#Ephafarma

,Nome,Governanca_B3,Previsao,Entrega,Ano_prev,Mes_prev,Dia_prev
1,ROMI,NM,2022-07-19,2022-07-19,2022,7,19
2,WEG,NM,2022-07-20,2022-07-20,2022,7,20
3,NEOENERGIA,NM,2022-07-26,2022-07-26,2022,7,26
4,CARREFOUR BR,NM,2022-07-26,2022-07-26,2022,7,26
5,SUZANO S.A.,NM,2022-07-27,2022-07-27,2022,7,27


In [15]:
#dt.time.time()
#a=DF2tri['Previsao']-hoje
DF2tri.count()

Nome             287
Governanca_B3    268
Previsao         287
Entrega          281
Ano_prev         287
Mes_prev         287
Dia_prev         287
dtype: int64

In [16]:
#Vamos criar um dicionário com a distribuição de datas para cada resultado
#manipulador=DF2tri.loc[DF2tri["Mes_prev"]==7].count()

mes_unico=DF2tri['Mes_prev'].unique()
ano_unico=DF2tri['Ano_prev'].unique()
dist_datas={}

for i in mes_unico:
    for b in ano_unico:
        manipulador=DF2tri.query(f"Mes_prev=={i} & Ano_prev=={b}")
        if manipulador.shape[0]==0:
            continue
            #manipulador=DF2tri.loc[DF2tri["Mes_prev"]==i].count()
        else:
            dist_datas[str(i)+'/'+str(b)]=manipulador.shape[0]

dist_datas


{'7/2022': 27,
 '8/2022': 251,
 '9/2022': 1,
 '10/2022': 3,
 '11/2022': 4,
 '2/2023': 1}

In [17]:
#pd.to_excel("calendario_bolsa.xltx",DF2tri)
#DF2tri.to_excel("calendario_bolsa.xlsx")

# Nessa parte, usaremos uma nova base com informações de setores providas pela B3 para classificarmos as empresas.

In [18]:
import zipfile
# ver documentação da Lib zipfile sempre que precisar: https://docs.python.org/3/library/zipfile.html

In [19]:
#Endereço na B3
file=r"https://bvmf.bmfbovespa.com.br/InstDados/InformacoesEmpresas/ClassifSetorial.zip"

#vamos abrir um arquivo Zip e vamos descompactá-lo
#sabemos que nessa pasta da B3 existe apenas uma planilha com a classificação setorial
name="ClassifSetorial.zip"#o nome do arquivo zip que vem da B3
with zipfile.ZipFile(name, 'r') as myzip:
        arquivo_listagem=myzip.namelist()
        myzip.extract(arquivo_listagem[0], path=None, pwd=None)

#depois de descompactar abriremos um dataframe com o arquivo da B3
base_setorial=pd.read_excel(arquivo_listagem[0])

In [20]:
nome=arquivo_listagem[0]
atualizacao=nome[12:22]
print(f"Última atualização setorial da B3: {atualizacao}")

Última atualização setorial da B3: 24-06-2022


Vamos dar uma purificada nesta base.

In [23]:
print(f"Quantidade inicial: {base_setorial.shape[0]}")
base_setorial.dropna(axis=0,how="all",inplace=True) #vamos remover linhas vazias
print(f"Quantidade final: {base_setorial.shape[0]}")

#como  removemos linhas vazias bagunçamos com o index do dataframe.
#Vamos então arrumar isso, refazendo o index
tam=base_setorial.shape[0]
base_setorial.index=range(0,tam,1)

#Vamos arrumar o título do dataframe também
titulo=["linha_0","linha_1","linha_2","linha_3","linha_4"]
base_setorial.columns=titulo

#pronto, agora temos uma base com o index arrumado e livre de linhas 100% vazias
base_setorial.head(10)


Quantidade inicial: 600
Quantidade final: 600


,linha_0,linha_1,linha_2,linha_3,linha_4
0,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,NaN
1,NaN,NaN,NaN,CÓDIGO,SEGMENTO
2,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
3,NaN,NaN,3R PETROLEUM,RRRP,NM
4,NaN,NaN,COSAN,CSAN,NM
5,NaN,NaN,DOMMO,DMMO,NaN
6,NaN,NaN,ENAUTA PART,ENAT,NM
7,NaN,NaN,PET MANGUINH,RPMG,NaN
8,NaN,NaN,PETROBRAS,PETR,N2
9,NaN,NaN,PETRORECSA,RECV,NM


In [31]:
'''
#Vamos ccriar um dataframe que receberá a listagem das empresas de forma mais limpa e legível.
#base_setorial.drop([0,1], axis=0, inplace=True)
alvo=base_setorial.iloc[0,:]
for i in range(base_setorial.shape[0]):
        comparador=base_setorial.iloc[i,:]
        if alvo==comparador:
            print(i)
            break
'''

'\n#Vamos ccriar um dataframe que receberá a listagem das empresas de forma mais limpa e legível.\n#base_setorial.drop([0,1], axis=0, inplace=True)\nalvo=base_setorial.iloc[0,:]\nfor i in range(base_setorial.shape[0]):\n        comparador=base_setorial.iloc[i,:]\n        if alvo==comparador:\n            print(i)\n            break\n'

In [27]:
#apagando as linhas sujas
#base_setorial.drop([0,5], axis=0, inplace=True)
titulos_1=base_setorial.iloc[5,:3]
titulos_2=base_setorial.iloc[6,3:]
base_setorial_B3=pd.DataFrame()

In [28]:
lista=[1,2,3,4]
base_setorial_B3["Setor"]=lista
base_setorial_B3["Segmento"]=lista
base_setorial_B3

titulo=["Setor","Subsetor","Segmento","Listagem_Codigo","Listagem_Governanca"]

In [30]:
'''
base_setorial.head(10)

#construiremos uma DataFrime com os dados lipos das empresas
DF_setorial_B3=pd.DataFrame({
    'Setor':None,
    'Subsetor':None,
    'Segmento':None,
    'Listagem_Codigo':None,
    'Listagem_governanca':None})
DF_setorial_B3
'''

"\nbase_setorial.head(10)\n\n#construiremos uma DataFrime com os dados lipos das empresas\nDF_setorial_B3=pd.DataFrame({\n    'Setor':None,\n    'Subsetor':None,\n    'Segmento':None,\n    'Listagem_Codigo':None,\n    'Listagem_governanca':None})\nDF_setorial_B3\n"

# Vamos carregar as informações do calendário de resultados em um banco de dados.

In [32]:
import sqlite3
import random
import datetime
import matplotlib.pyplot as plt
%matplotlib notebook
 
# Criando uma conexão
conn = sqlite3.connect('result_2T22.db')   

# Criando um cursor
c = conn.cursor()
 
# Função para criar uma tabela
def create_table():
    c.execute('CREATE TABLE IF NOT EXISTS produtos(id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, date TEXT, '\
              'prod_name TEXT, valor REAL)')
     
# Função para inserir uma linha
def data_insert():
    c.execute("INSERT INTO produtos VALUES(now, 'Teclado', 130 )")
    conn.commit()
    c.close()
    conn.close()
    
# Usando variáveis para inserir dados    
def data_insert_var():
    new_date = datetime.datetime.now()
    new_prod_name = 'monitor'
    new_valor = random.randrange(50,100)
    c.execute("INSERT INTO produtos (date, prod_name, valor) VALUES (?, ?, ?, ?)", 
              (new_date, new_prod_name, new_valor))
    conn.commit()
    
# Leitura de dados
def leitura_todos_dados():
    c.execute("SELECT * FROM PRODUTOS")
    for linha in c.fetchall():
        print(linha)
        
# Leitura de registros específicos
def leitura_registros():
    c.execute("SELECT * FROM PRODUTOS WHERE valor > 60.0")
    for linha in c.fetchall():
        print(linha)      
        
# Leitura de colunas específicos
def leitura_colunas():
    c.execute("SELECT * FROM PRODUTOS")
    for linha in c.fetchall():
        print(linha[3])  
        
# Update
def atualiza_dados():
    c.execute("UPDATE produtos SET valor = 70.00 WHERE valor > 80.0")
    conn.commit()
    
# Delete
def remove_dados():
    c.execute("DELETE FROM produtos WHERE valor = 62.0")
    conn.commit()